In [20]:
# 打开图像文件
image_path = './public/00236-3304752085.png'  # 替换为你的图片路径

In [22]:
from PIL import Image

img = Image.open(image_path)
type(img.info)



dict

In [17]:
tags = {
  'tags': "1girl",
}
img.save('./public/t1.jpg', "JPEG", quality=95, optimize=True, info=tags)

In [18]:
image_path = './public/t1.jpg'
img = Image.open(image_path)
img.info

{'jfif': 257, 'jfif_version': (1, 1), 'jfif_unit': 0, 'jfif_density': (1, 1)}

In [ ]:
from pyexiv2 import Image
from pathlib import Path

image_path = './public/00236-3304752085.png' 
i = Image(Path(image_path))
i.read_comment()
i.close()

AttributeError: 'WindowsPath' object has no attribute 'encode'